In [4]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import

from keras.engine import Layer, InputSpec
from keras import initializations, regularizers
from keras import backend as K
from keras import activations

from theano import tensor as T
from theano import scan 
import numpy as np

class SecondaryStatistic(Layer):
    ''' This layer shall compute the image secondary statistics and
    output the probabilities. Two computation in one single layer.
    Input to this layer shall come from an output of a convolution layer.
    To be exact, from Convolution2D
    Thus shall be in format of

    # Input shape
        (samples, nb_filter, rows, cols)

    # Output shape
        3D tensor with
            (samples, out_dim, out_dim)
        This is just the 2D covariance matrix for all samples feed in.

    # Arguments
        out_dim         weight matrix, if none, make it align with
        weights         initial weights.
        W_regularizer   regularize the weight if possible in future
        init:           initialization of function.
        activation      test activation later


    '''

    def __init__(self, out_dim=None,
                 init='glorot_uniform', activation='linear', weights=None,
                 W_regularizer=None, dim_ordering='default',  **kwargs):
        #TODO Finish this
        self.out_dim = out_dim
        
        # input parameter preset
        self.nb_filter = 0
        self.cols = 0
        self.rows = 0
        self.nb_samples = 0
        
        self.activition = activations.get(activation)

        self.init = initializations.get(init, dim_ordering = dim_ordering)
        self.initial_weights = weights
        self.W_regularizer = regularizers.get(W_regularizer)
        self.dim_ordering='th'
        
        
        self.input_spec = [InputSpec(ndim=4)]
        super(SecondaryStatistic, self).__init__(**kwargs)

    def build(self, input_shape):
        # TODO Weight is not supported at this moment, just write them as exp
        print(len(input_shape))
        
        
        
        if self.dim_ordering == 'th':
            
            self.cov_dim = input_shape[1]
            
            self.nb_samples = input_shape[0]
            self.nb_filter = input_shape[1]
            self.rows = input_shape[2]
            self.cols = input_shape[3]
            
            if self.out_dim is not None:
                self.W_shape = (self.out_dim, self.out_dim)
                if self.initial_weights is not None:
                    self.set_weights(self.initial_weights)
                    del self.initial_weights
                else:
                    self.W = self.init(self.W_shape, name='{}_W'.format(self.name))
                self.trainable_weights = [self.W]
            else:
                # No input parameters, set the weights to identity matrix
                self.W_shape =(self.cov_dim,self.cov_dim)
                print(self.W_shape)
                self.out_dim = self.cov_dim
                self.W = K.eye(self.cov_dim, name='{}_W'.format(self.name))
                self.non_trainable_weights = [self.W]
        else:
            raise Exception('Invalid dim_ordering: ' + self.dim_ordering
                            +' tensorflow not supported')

        self.built = True

    def get_output_shape_for(self, input_shape):
        # TODO Check the validity.

        return (input_shape[0], self.out_dim,self.out_dim)

    def call(self, x, mask=None):

        print(type(x)) # Confirm the type of x is indeed tensor4D
        
        # TODO Compute the mean vector
        # Pesudo data creation
        # x = np.random.rand(*shape)
        
        # Step 1: reshape the 3D array into 2D array
        
        # type (theano.config.floatX, matrix)
        # TODO Compute the covariance matrix Y, by sum( <x_ij - x, x_ij.T - x.T> )
        components,updates = scan(fn=lambda tx: self.calculate_covariance(tx),
                                         outputs_info=None,
                                         sequences=[x],
                                         non_sequences=None)
        print(components.type)
        print(components.eval().shape)
        print(components.eval())
        return components

    def get_config(self):
        config = {'init': self.init.__name__,
                  'activation': self.activation.__name__,
                  'dim_ordering': self.dim_ordering,
                  'W_regularizer': self.W_regularizer.get_config() if self.W_regularizer else None,
                  }
        base_config = super(SecondaryStatistic, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def reshape_tensor2d(self,x):
        # given a 3D tensor, reshape it to 2D.
        return K.reshape(K.flatten(x.T),
                         (self.cols*self.rows,
                          self.nb_filter))
        
    def calculate_covariance(self, x):
        # Input shall be 3D tensor (nb_filter,ncol,nrow)
        # Return just (nb_filter, nb_filter)
        tx = self.reshape_tensor2d(x)
        # Calcualte the covariance 
        tx_mean = K.mean(tx,axis=0)
        # return tx_mean
        tx_normal = tx - tx_mean
        # return tx_normal
        tx_cov = K.dot(tx_normal.T,tx_normal) / (self.cols*self.rows - 1)
        return tx_cov

In [73]:
# Test layer.

# data creation
# create only 1 sample from conv layer.

shape = (3,4,5)
mat = np.array(range(1,61))
mat = mat.reshape(shape)
mat = np.expand_dims(mat,0)

# For random test
# shape = (30,20,100,500)
# mat = np.random.rand(*shape)

ipt_shape = mat.shape
print(mat.shape)
# create the corresnponding 4D tensor
ten4d = K.variable(mat)
print(ten4d.type)

sndlayer = SecondaryStatistic()
sndlayer.build(ipt_shape)
res = sndlayer.call(ten4d)

(1, 3, 4, 5)
TensorType(float32, 4D)
4
(3, 3)
<class 'theano.tensor.sharedvar.TensorSharedVariable'>
TensorType(float32, matrix)


(1, 20)
[[ 21.  26.  31.  36.  22.  27.  32.  37.  23.  28.  33.  38.  24.  29.
   34.  39.  25.  30.  35.  40.]]


In [50]:
import numpy

coefficients = theano.tensor.vector("coefficients")
x = T.scalar("x")

max_coefficients_supported = 10000

# Generate the components of the polynomial
components, updates = theano.scan(fn=lambda coefficient, power, free_variable: coefficient * (free_variable ** power),
                                  outputs_info=None,
                                  sequences=[coefficients, theano.tensor.arange(max_coefficients_supported)],
                                  non_sequences=x)
# Sum them up
polynomial = components.sum()

# Compile a function
calculate_polynomial = theano.function(inputs=[coefficients, x], outputs=polynomial)

# Test
test_coefficients = numpy.asarray([1, 0, 2], dtype=numpy.float32)
test_value = 3
print(calculate_polynomial(test_coefficients, test_value))
print(1.0 * (3 ** 0) + 0.0 * (3 ** 1) + 2.0 * (3 ** 2))

NameError: name 'theano' is not defined

In [88]:
sum_mat = K.sum(mat, axis=1)
sum_mat.eval()
print(mat.shape)
print(sum_mat.eval().shape)
sum2 = K.sum(sum_mat,axis=0)
print(sum2.eval().shape)
# mat3 = K.sum(sum2, axis=0)
print(K.sum(sum2,axis=0).eval().shape)
K.variable(np.ndarray((10,),), dtype='float32')

(1, 3, 4, 5)
(1, 4, 5)
(4, 5)
(5,)


<TensorType(float32, vector)>

In [8]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import

from keras.engine import Layer, InputSpec
from keras import initializations, regularizers
from keras import backend as K
from keras import activations

from theano.tensor.elemwise import *
from theano import scalar


class WeightedVectorization(Layer):
    ''' Probability weighted vector layer for secondary image statistics
    neural networks. It is simple at this time, just v_c.T * Cov * v_c, with
    basic activitation function such as ReLU, softmax, thus the

    '''


    def __init__(self, output_dim, input_dim=None, init='glorot_uniform', activation='linear', weights=None,
                 # W_regularizer=None, b_regularizer=None, activity_regularizer=None,
                 # W_constraint=None, b_constraint=None,
                 bias=False,  **kwargs):
        self.init = initializations.get(init)
        self.activation = activations.get(activation)
        self.input_dim = input_dim      # Squared matrix input, as property of cov matrix
        self.output_dim = output_dim    # final classified categories number

        self.bias = bias
        self.initial_weights = weights

        super(WeightedVectorization, self).__init__(**kwargs)

    def build(self, input_shape):
        # 3D tensor (nb_samples, n_cov, n_cov)
        assert input_shape and len(input_shape) == 3
        assert input_shape[1] == input_shape[2]

        input_dim = input_shape[1]

        self.W = self.init((input_dim, self.output_dim), name='{}_W'.format(self.name))

        if self.bias:
            self.b = K.zeros((self.output_dim,), name='{}_b'.format(self.name))
            self.trainable_weights = [self.W, self.b]
        else:
            self.trainable_weights = [self.W]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        '''
        The calculation of call function is not trival.
        sum( self.W .* ( x * self.W) ) along axis 1
        :param x:
        :param mask:
        :return: final output vector with w_i^T * W * w_i as item i, and propagate to all
            samples. Output Shape (nb_samples, vector c)
        '''
        print('Weights ', self.W)
        print('Input ', x)
        
        output = K.sum(Elemwise(scalar_op=scalar.mul)(self.W, K.dot(x, self.W)), axis=1)
        if self.bias:
            output += self.b
        return self.activation(output)

    def get_output_shape_for(self, input_shape):
        # 3D tensor (nb_samples, n_cov, n_cov)
        '''
        :param input_shape: 3D tensor where item 1 and 2 must be equal.
        :return: (nb_samples, number C types)
        '''
        assert input_shape and len(input_shape) == 3
        assert input_shape[1] == input_shape[2]
        return (input_shape[0], self.output_dim)

    def get_config(self):
        config = {'output_dim': self.output_dim,
                  'init': self.init.__name__,
                  'activation': self.activation.__name__,
                  # 'W_regularizer': self.W_regularizer.get_config() if self.W_regularizer else None,
                  # 'b_regularizer': self.b_regularizer.get_config() if self.b_regularizer else None,
                  # 'activity_regularizer': self.activity_regularizer.get_config() if self.activity_regularizer else None,
                  # 'W_constraint': self.W_constraint.get_config() if self.W_constraint else None,
                  # 'b_constraint': self.b_constraint.get_config() if self.b_constraint else None,
                  'bias': self.bias,
                  'input_dim': self.input_dim}
        base_config = super(WeightedVectorization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
# Test for the final cov to prob layer.

## Generate cov matrix inputs.
cov = np.random.rand(2,4,4)
# 4D corresponding tensor.
ten_cov = K.variable(cov, dtype='float32')
# Generate output dim
C = 3


# Construct layer
prob = WeightedVectorization(C)
prob.build(cov.shape)
result = prob.call(cov)
print(result.eval().shape)

('Weights ', weightedprobability_3_W)
('Input ', array([[[ 0.26807582,  0.90746333,  0.08264032,  0.11614684],
        [ 0.24163645,  0.02988074,  0.1308075 ,  0.22704176],
        [ 0.57694447,  0.37901564,  0.22230386,  0.12285798],
        [ 0.4671178 ,  0.67941401,  0.53050448,  0.47304899]],

       [[ 0.91290742,  0.70926575,  0.22687191,  0.78384609],
        [ 0.03261848,  0.42249514,  0.9722906 ,  0.12957048],
        [ 0.5674235 ,  0.82629979,  0.86577628,  0.97576602],
        [ 0.54057234,  0.57597883,  0.31316738,  0.39361311]]]))


(2, 3)
